# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv(r'C:\Users\tiago\Documents\DATA_SCIENCE\EBAC\M05\3a46234d29\Tarefa1\SINASC_RO_2019.csv', encoding='utf-8' )
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [3]:
# 2) Contando valores missing por variavel no dataframe
total_missim_variavel = sinasc.isna().sum()
total_missim_variavel

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [4]:
# 3) Selecionando apenas colunas interessadas
sinasc_filtrado = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_filtrado

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [5]:
# 3) Contando novamente os valores missing
total_missim_filtrado = sinasc_filtrado.isna().sum()
total_missim_filtrado

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [6]:
(sinasc_filtrado.isna().sum()/len(sinasc_filtrado))*100

LOCNASC       0.000000
IDADEMAE      0.000000
ESTCIVMAE     1.172858
ESCMAE        1.154358
QTDFILVIVO    5.819890
GESTACAO      4.558236
GRAVIDEZ      0.292289
CONSULTAS     0.000000
APGAR5        0.381086
dtype: float64

In [7]:
# 4) Calculando o numero de linhas do dataframe
linhas, _= sinasc_filtrado.shape

print(linhas)


27028


In [8]:
# 4) Calculando a quantidade valores missing da coluna 'APGAR5'
sinasc_filtrado['APGAR5'].isna().sum()

103

In [9]:
# 4) Removendo dados ausentes da coluna da coluna 'APGAR5'
sinasc_2 = sinasc_filtrado[~sinasc_filtrado['APGAR5'].isna()]
sinasc_2

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [10]:
sinasc_2.shape

(26925, 9)

In [11]:
sinasc_2.dtypes

LOCNASC         int64
IDADEMAE        int64
ESTCIVMAE     float64
ESCMAE         object
QTDFILVIVO    float64
GESTACAO       object
GRAVIDEZ       object
CONSULTAS       int64
APGAR5        float64
dtype: object

In [12]:
sinasc_2.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [15]:
# 5) seu código aqui
sinasc_2['ESTCIVMAE'].fillna(9, inplace=True)
sinasc_2['CONSULTAS'].fillna(9, inplace=True)

In [16]:
sinasc_2.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [17]:
# 6) Verificando a quantidade de valores faltantes
sinasc_2['QTDFILVIVO'].isna().sum()

1566

In [18]:
# 6) Substituindo valores faltantes por 0
sinasc_2['QTDFILVIVO'].fillna(0, inplace=True)

In [19]:
# 6) Verificando a quantidade de valores faltantes
sinasc_2['QTDFILVIVO'].isna().sum()

0

In [20]:
# 7) Irei alterar apenas as colunas 'ESCMAE' e 'GRAVIDEZ' a coluna ESCMAE onde for NaN sera substitudo por nenhuma escolaaridade
# considerada no dicionario do SINASC e a coluna GRAVIDEZ sera considerada com valor 9 ou seja informaçao ingnorada
sinasc_2['ESCMAE'].fillna(1, inplace=True)
sinasc_2['GRAVIDEZ'].fillna(9, inplace=True)

8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:

Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.
Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

In [22]:
# 8) seu código aqui
sinasc_2.loc[(sinasc_2['APGAR5'] >= 8) & (sinasc_2['APGAR5'] <=10), 'cat_apgar5'] = 'normal'
sinasc_2.loc[(sinasc_2['APGAR5'] >= 6) & (sinasc_2['APGAR5'] <=7), 'cat_apgar5'] = 'asfixia leve'
sinasc_2.loc[(sinasc_2['APGAR5'] >= 4) & (sinasc_2['APGAR5'] <=5), 'cat_apgar5'] = 'asfixia moderada'
sinasc_2.loc[(sinasc_2['APGAR5'] >= 0) & (sinasc_2['APGAR5'] <=3), 'cat_apgar5'] = 'asfixia severa'

In [24]:
dummies = pd.get_dummies(sinasc_2['cat_apgar5'])
dummies

,asfixia leve,asfixia moderada,asfixia severa,normal
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
27023,0,0,0,1
27024,0,0,0,1
27025,0,0,0,1
27026,0,0,0,1


In [25]:
dummies.value_counts()

asfixia leve  asfixia moderada  asfixia severa  normal
0             0                 0               1         26463
1             0                 0               0           320
0             0                 1               0            74
              1                 0               0            68
dtype: int64

In [26]:
dummies['asfixia leve'].value_counts()


0    26605
1      320
Name: asfixia leve, dtype: int64

In [27]:
dummies['asfixia moderada'].value_counts()


0    26857
1       68
Name: asfixia moderada, dtype: int64

In [28]:
dummies['asfixia severa'].value_counts()


0    26851
1       74
Name: asfixia severa, dtype: int64

In [29]:
dummies['normal'].value_counts()

1    26463
0      462
Name: normal, dtype: int64

In [30]:
novo_sinasc = sinasc_2.copy()
novo_sinasc = pd.concat([novo_sinasc, dummies], axis=1)

In [31]:
novo_sinasc.head(n=60)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,cat_apgar5,asfixia leve,asfixia moderada,asfixia severa,normal
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal,0,0,0,1
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal,0,0,0,1
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal,0,0,0,1
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal,0,0,0,1
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal,0,0,0,1
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal,0,0,0,1
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal,0,0,0,1
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,normal,0,0,0,1
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0,normal,0,0,0,1
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0,normal,0,0,0,1


In [32]:
# 9) seu código aqui
colunas = sinasc_2.columns
colunas

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'cat_apgar5'],
      dtype='object')

In [33]:
colunas_snake_case = []
for i in colunas:
    colunas_snake_case.append(i.lower())

In [34]:
colunas_snake_case

['locnasc',
 'idademae',
 'estcivmae',
 'escmae',
 'qtdfilvivo',
 'gestacao',
 'gravidez',
 'consultas',
 'apgar5',
 'cat_apgar5']

In [35]:
dff = sinasc_2.copy()
dff

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,cat_apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [36]:
dff.columns = colunas_snake_case

In [37]:
dff

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,cat_apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [38]:
# 9) Renomeando as colunas conforme o enunciado
colunas_renomeadas = ['local_nascimento', 'idade_mae', 'estado_civil_mae', 'escolaridade_mae', 'quantidade_filhos_vivos',
                      'semanas_gestacao', 'tipo_gravidez', 'numero_consultas', 'apgar_quinto_minuto', 'cat_apgar5']

In [39]:
sinasc_2.columns = colunas_renomeadas
sinasc_2

,local_nascimento,idade_mae,estado_civil_mae,escolaridade_mae,quantidade_filhos_vivos,semanas_gestacao,tipo_gravidez,numero_consultas,apgar_quinto_minuto,cat_apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
